In [ ]:
import numpy as np
import cv2 as cv

# Читать видео
cap = cv.VideoCapture('D:/vsprojects/images/vtest.avi')
# Фоновая модель извлечения
fgbg = cv.createBackgroundSubtractorMOG2(
    history=500, varThreshold=100, detectShadows=False)

# Предварительная обработка
def process(image, opt=1):
    mask = fgbg.apply(frame)
    line = cv.getStructuringElement(cv.MORPH_RECT, (1, 5), (-1, -1))
    # Открытая операция
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, line)
    cv.imshow("mask", mask)
    # Контур извлечения, найти самый большой контур
    contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    for c in range(len(contours)):
        area = cv.contourArea(contours[c])
        if area < 100:
            continue
        # Извлечь прямоугольник
        rect = cv.minAreaRect(contours[c])
        box = cv.boxPoints(rect)
        box = np.int0(box)
        # Прямоугольник
        cv.drawContours(image, [box], 0, (0, 0, 255), 2)
        # Центральная точка
        cv.circle(image, (np.int32(rect[0][0]), np.int32(rect[0][1])), 2, (255, 0, 0), 2, 8, 0)
    return image, mask


while True:
    ret, frame = cap.read()
    cv.imwrite("D:/input.png", frame)
    cv.imshow('input', frame)
    result, m_ = process(frame)
    cv.imshow('result', result)
    k = cv.waitKey(50) & 0xff
    if k == 27:
        cv.imwrite("D:/result.png", result)
        cv.imwrite("D:/mask.png", m_)

        break
cap.release()
cv.destroyAllWindows()